In [18]:
from functools import reduce
import gc
import numpy as np

"""STVM 계산(3차로 점선) + 지상부 추가"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 5400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 60

# 지상부_진출 검지기
exit_line = 265
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(df):
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도
def density_mean(df):
    density_mean_df = df.assign(K = df["V_count"] * 12 / df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):

    # 진입부 검지기 선정
    measurement = enter_line

    # TimeGroup별 중차량 갯수 집계
    heavy_df = (
        original_df[
            (original_df["New_Measurement"] == measurement) &
            (original_df["Vehicle type"].isin([630,640,650]))
        ]
        .groupby("TimeGroup")
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        original_df[original_df["New_Measurement"] == measurement]
        .groupby("TimeGroup")
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(heavy_df, total_df, on="TimeGroup", how="left")

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    # 실측용량 C
    max_capacity = 2200
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        original_df[original_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    entry_saturation_df["Phi_진입"] = 0
    return entry_saturation_df

# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0.2값으로 fix
    main_df["RFR"] = 0.2
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = original_df[original_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    exit_df = original_df[original_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )
    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
    merged_counts["1-F(outrate)"] = 1 - merged_counts["F(outrate)"]
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "rate"]], on="TimeGroup")
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )

    merged["STVM"] = (
        weights["w1"] * merged["delta_V"] +
        weights["w2"] * merged["delta_K"] +
        weights["w3"] * merged["rate"] +
        weights["w4"] * merged["Phi_진입"] +
        weights["w5"] * merged["RFR"] +
        weights["w6"] * merged["1-F(outrate)"]
    )
    """
    merged_df = merged.pivot(index='TimeGroup',
                columns='New_Measurement',
                values='STVM')
    merged_df = merged_df.sort_index().sort_index(axis=1)
    modify_stvm_df = modify_frame(merged_df)
    """
    return merged


def calculate_z_score(stvm_df):
    # 평균
    mean_stvm = stvm_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = stvm_df["STVM"].std(skipna=True)

    # Z-Score 계산
    stvm_df["Z-Score"] = (stvm_df["STVM"] - mean_stvm) / std_stvm
    z_max = stvm_df["Z-Score"].max()
    z_min = stvm_df["Z-Score"].min()

    stvm_df["환산점수"] = stvm_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(stvm_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")

    return stvm_df

def modify_frame(z_score_df):
    result_df = z_score_df.copy()
    result_df = result_df.drop(columns=list(range(1,6)) + list(range(276,281)))

    columns = result_df.columns.tolist()

    new_columns = [-5, -4, -3, -2, -1]
    # 나머지 (11번 이후) → 1부터 순차 번호
    new_columns += list(range(1, len(columns) - 5 + 1))
    result_df.columns = new_columns
    return result_df

def calculate_avg(df):

    def avg_range(df, low, high):
        cols = [c for c in df.columns if low <= c <= high]
        return df[cols].mean(axis=1) if cols else np.nan

    avg_df = pd.DataFrame(index=df.index)
    avg_df["지상부_진입"] = avg_range(df, -5, -1)
    avg_df["진입부"] = avg_range(df, 1, 31)
    avg_df["본선부"] = avg_range(df, 31, 231)
    avg_df["진출부"] = avg_range(df, 231, 261)
    avg_df["지상부_진출"] = avg_range(df, 261, 266)

    return avg_df

def merged_varible(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):
    # New_Measurement 별 피봇
    pivot_speed_df = speed_df.pivot(index='TimeGroup',
                columns='New_Measurement',
                values='delta_V')
    pivot_speed_df = pivot_speed_df.sort_index().sort_index(axis=1)

    pivot_density_df = density_df.pivot(index='TimeGroup',
            columns='New_Measurement',
            values='delta_K')
    pivot_density_df = pivot_density_df.sort_index().sort_index(axis=1)

    speed_new_df = modify_frame(pivot_speed_df)
    density_new_df = modify_frame(pivot_density_df)

    avg_speed_df = calculate_avg(speed_new_df)
    avg_density_df = calculate_avg(density_new_df)

    merged_df_list = [avg_speed_df, avg_density_df, heavy_df[["TimeGroup", "rate"]], entry_saturation_df[["TimeGroup", "Phi_진입"]], rfr_df[["TimeGroup", "RFR"]], normality_df[["TimeGroup", "1-F(outrate)"]]]

    merged_df = reduce(lambda left, right: pd.merge(left, right, on="TimeGroup", how="left"), merged_df_list)

    return merged_df


def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")


def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\mer파일\1750"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()

for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)
                display("original_df : ", original_df)

                # 평균속도
                speed_df = speed_mean(original_df)

                # 밀도
                density_df = density_mean(speed_df)

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)

                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)

                # 진출 정상성
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #save_to_excel(stvm_df, folder_path, "STVM", i)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)

                # 결과 정리
                result_df = modify_frame(z_score_df)
                display(result_df)

                # 변수별 데이터 모음(STVM 결과랑 상관 X)
                #varible_df = merged_varible(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)

                save_to_excel(result_df, folder_path, "환산점수", i)
                #save_to_excel(varible_df, folder_path, "세부항목", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

'original_df : '

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
0,10005,1800.07,-1.0,2603,100,0,110.9,4.21,NaN,5,1800~2100
1,10011,1800.02,-1.0,2584,100,0,104.6,3.75,NaN,11,1800~2100
2,10125,1800.01,-1.0,1989,100,0,95.3,4.01,NaN,125,1800~2100
3,10157,1800.05,-1.0,1791,100,0,98.1,4.01,NaN,157,1800~2100
4,10170,1800.02,-1.0,1733,300,0,100.0,12.40,NaN,170,1800~2100
...,...,...,...,...,...,...,...,...,...,...,...
1458161,30202,5399.93,-1.0,6798,100,0,104.4,4.76,NaN,202,5100~5400
1458162,30217,5399.93,-1.0,6729,100,0,95.3,4.76,NaN,217,5100~5400
1458163,30221,5399.93,-1.0,6720,100,0,101.1,4.64,NaN,221,5100~5400
1458164,30236,5399.99,-1.0,6629,100,0,110.5,4.76,NaN,236,5100~5400


,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,62.762172,63.183782,63.351281,63.016176,62.348462,62.929414,62.758339,62.757059,63.447422,62.928835,...,62.579539,62.660134,63.014401,63.014356,63.372443,62.841135,63.014357,62.835687,63.014475,63.375134
2100~2400,67.314207,67.311148,67.019246,67.809936,68.320556,67.805234,67.603537,67.803546,67.503651,68.006239,...,67.604963,67.790897,68.055530,67.515429,67.191537,67.701380,67.694508,67.341145,67.339246,67.694760
2400~2700,64.480875,64.401674,64.317937,63.848484,64.000582,63.918615,64.239400,64.481602,64.480137,64.239883,...,64.317503,64.161409,63.848482,64.396867,64.250273,64.477489,64.317493,64.475268,64.394273,63.853417
2700~3000,62.173798,62.427758,62.944106,62.854464,62.515341,62.764872,62.513287,62.179917,62.344974,62.428730,...,62.513514,62.842195,62.676372,62.759372,62.915841,62.674120,62.119743,62.513762,62.432835,62.674090
3000~3300,65.113847,64.624026,64.623587,65.021958,64.701103,64.468594,65.021960,65.115676,64.310723,64.461157,...,65.126401,64.429361,65.127128,65.442163,65.024254,64.524561,65.126848,64.168879,64.599329,65.128954
3300~3600,60.905709,60.818674,60.566497,60.233001,60.566823,61.154711,60.483625,60.400404,61.149622,60.893418,...,60.171045,60.646412,60.486253,60.165066,60.413264,60.407677,60.566470,60.648123,60.975449,60.808485
3600~3900,68.903501,69.197275,68.790068,69.397287,69.293096,69.082674,70.007594,70.033528,69.706239,69.398811,...,70.106476,69.797770,70.048136,69.600006,69.220181,69.696187,69.313143,70.021048,69.309488,68.633567
3900~4200,62.319671,62.090251,62.542047,62.239019,62.090774,62.312438,62.016561,62.163815,61.500617,61.865354,...,62.090304,62.250844,62.089968,62.329045,62.253885,61.863107,62.092934,61.693073,62.171045,62.651871
4200~4500,63.916825,64.221386,63.841143,63.916716,63.916638,63.687495,63.761587,63.604161,64.229065,64.227473,...,64.147694,63.993919,63.606871,63.992803,64.305360,64.224328,64.451930,64.069104,63.917224,63.841191


환산점수_1.xlsx 생성 완료


'original_df : '

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
0,10013,1800.04,-1.0,2472,630,0,102.8,6.67,NaN,13,1800~2100
1,10020,1800.01,-1.0,2456,100,0,106.5,4.21,NaN,20,1800~2100
2,10027,1800.09,-1.0,2415,100,0,103.4,4.01,NaN,27,1800~2100
3,10095,1800.01,-1.0,2049,100,0,86.1,4.76,NaN,95,1800~2100
4,10151,1800.01,-1.0,1706,630,0,100.4,6.67,NaN,151,1800~2100
...,...,...,...,...,...,...,...,...,...,...,...
1402535,30180,5400.00,-1.0,6592,300,0,100.0,12.40,NaN,180,NaN
1402536,30229,5399.93,-1.0,6345,100,0,103.0,4.61,NaN,229,5100~5400
1402537,30230,5399.91,-1.0,6284,630,0,99.8,6.67,NaN,230,5100~5400
1402538,30240,5399.95,-1.0,6320,100,0,21.2,4.76,NaN,240,5100~5400


,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,61.161517,60.912685,60.748902,60.094335,60.746229,61.331199,60.502358,60.580983,61.164174,60.664413,...,61.615882,61.930167,61.633809,60.676895,63.903926,61.604085,61.217317,61.016867,60.850556,61.493902
2100~2400,65.949572,65.947597,66.221316,66.771076,66.130434,65.677903,66.772428,66.130316,65.675734,66.404266,...,66.413047,66.500324,65.663530,66.234003,70.041731,66.969201,65.923681,66.177737,66.057020,65.665086
2400~2700,62.732828,63.347830,63.192640,63.116227,63.500505,63.115824,62.889657,63.420220,63.266790,62.891410,...,62.353135,62.509772,63.278105,62.834948,67.079638,63.615335,63.216998,62.845623,63.378874,63.201363
2700~3000,58.042065,57.902334,57.443072,57.441884,57.221888,57.630683,57.906423,57.561544,57.836095,57.969285,...,57.635563,57.257291,57.350253,58.457689,60.891117,58.285786,58.093518,57.951028,57.440904,57.585693
3000~3300,61.236593,61.000836,61.864766,61.785397,61.793294,62.012226,61.092960,60.942266,61.003957,60.683474,...,61.323114,61.630561,61.829806,60.604578,64.524361,62.287346,61.245053,61.411199,61.805762,61.717684
3300~3600,61.315604,61.167100,61.091619,61.091406,60.944136,60.867672,61.397638,61.314856,61.312443,61.463844,...,61.366715,61.329869,60.899530,61.355812,64.627729,61.835216,61.610268,60.476710,60.937397,61.186751
3600~3900,59.482030,59.481109,59.629330,59.703459,59.776944,59.556575,59.481152,59.703549,59.703120,59.629017,...,59.767310,59.696053,59.866288,59.764498,63.808710,60.381059,59.800990,60.106160,59.894217,59.706798
3900~4200,69.549554,69.372126,69.195181,69.106427,69.282349,69.192724,69.550450,69.727437,69.107280,69.104778,...,69.113531,69.169823,69.124604,68.951454,71.224630,70.097115,69.541144,68.969658,69.209608,69.468051
4200~4500,95.711405,95.881560,95.813074,95.610276,95.882457,95.914635,95.678340,95.609748,95.779280,95.882085,...,95.750526,95.993228,96.028932,96.164927,99.713722,96.131665,95.908325,95.892860,95.594645,95.618111


환산점수_2.xlsx 생성 완료
